In [1]:
"""
The script demonstrates a simple example of using ART with PyTorch. The example train a small model on the MNIST dataset
and creates adversarial examples using the Fast Gradient Sign Method. Here we use the ART classifier to train the model,
it would also be possible to provide a pretrained model to the ART classifier.
The parameters are chosen for reduced computational requirements of the script and not optimised for accuracy.
"""
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

from art.attacks import UniversalPerturbation
from art.classifiers import PyTorchClassifier
from art.utils import load_mnist

from torchvision import datasets, transforms
import torch
import numpy as np
from torch.utils.data import SubsetRandomSampler

from matplotlib import pyplot as plt

import torchvision.models as models
import torch.nn.init as init


In [2]:
np.random.seed(1)
torch.manual_seed(1)
torch.set_num_threads(4)
numberOfClasses = 2

class Fire(nn.Module):

    def __init__(self, inplanes, squeeze_planes,
                 expand1x1_planes, expand3x3_planes):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes,
                                   kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes,
                                   kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


class SqueezeNetCustom(nn.Module):

    def __init__(self, version='1_1', num_classes=2):
        super(SqueezeNetCustom, self).__init__()
        self.num_classes = num_classes
        if version == '1_0':
            self.features = nn.Sequential(
                nn.Conv2d(3, 96, kernel_size=7, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(96, 16, 64, 64),
                Fire(128, 16, 64, 64),
                Fire(128, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 32, 128, 128),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(512, 64, 256, 256),
            )
        elif version == '1_1':
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(64, 16, 64, 64),
                Fire(128, 16, 64, 64),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(128, 32, 128, 128),
                Fire(256, 32, 128, 128),
                nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
                Fire(256, 48, 192, 192),
                Fire(384, 48, 192, 192),
                Fire(384, 64, 256, 256),
                Fire(512, 64, 256, 256),
            )
        else:
            # FIXME: Is this needed? SqueezeNet should only be called from the
            # FIXME: squeezenet1_x() functions
            # FIXME: This checking is not done for the other models
            raise ValueError("Unsupported SqueezeNet version {version}:"
                             "1_0 or 1_1 expected".format(version=version))

        # Final convolution is initialized differently from the rest
        final_conv = nn.Conv2d(512, self.num_classes, kernel_size=1)
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            final_conv,
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if m is final_conv:
                    init.normal_(m.weight, mean=0.0, std=0.01)
                else:
                    init.kaiming_uniform_(m.weight)
                if m.bias is not None:
                    init.constant_(m.bias, 0)

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        x = torch.flatten(x, 1)
        return F.log_softmax(x, dim=1)

In [3]:
# Setp 1a: define data transform
data_transform = transforms.Compose([
        #transforms.RandomRotation(30),
        transforms.Resize((224,224)),
        #transforms.RandomCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),                     
        transforms.Normalize(                     
            mean=[0.485, 0.456, 0.406],               
            std=[0.229, 0.224, 0.225]                  
        )])



# Step 1: Load the dataset
breakhis = datasets.ImageFolder(root ='C:\\Users\\Florian\\Desktop\\Datensätze_ready\\BreakHis\\BreaKHis_v1\\', 
                                 transform=data_transform)


# split into train and test dataset
train_split = 0.8
dataset_size = len(breakhis) #for testing purpose set to 1000 - else set: len(trafficsign) 
indices = list(range(dataset_size))
split = int(np.floor(train_split * dataset_size))
np.random.shuffle(indices)
train_indices, test_indices = indices[:split], indices[split:]

#TODO ÄNDERN
#train set
dataset_loader_train = torch.utils.data.DataLoader(breakhis, sampler=SubsetRandomSampler(train_indices), batch_size=int(np.floor(train_split * dataset_size)))
dataiter = iter(dataset_loader_train)
images, labels = dataiter.next()

x_train = images.data.numpy()
y_train = np.zeros((len(labels.data.numpy()), len(breakhis.classes)))
y_train[np.arange(len(labels.data.numpy())), labels.data.numpy()] = 1

#test set
dataset_loader_test = torch.utils.data.DataLoader(breakhis, sampler=SubsetRandomSampler(test_indices), batch_size=(len(breakhis)-int(np.floor(train_split * dataset_size))))
dataiter = iter(dataset_loader_test)
images, labels = dataiter.next()
x_test = images.data.numpy()
y_test = np.zeros((len(labels.data.numpy()), len(breakhis.classes)))
y_test[np.arange(len(labels.data.numpy())), labels.data.numpy()] = 1

#filter indices to get only benign tumors to trick them all
indices = np.where(labels == 1)
x_adv = x_test[indices]
y_adv = y_test[indices]

indices = np.where(labels == 0)
x_benign = x_test[indices]
y_benign = y_test[indices]

In [4]:
# Step 2: Create the model
model = SqueezeNetCustom()

In [5]:
# Step 2a: Define the loss function and the optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [6]:
# Step 3: Create the ART classifier

classifier = PyTorchClassifier(model=model, loss=criterion, optimizer=optimizer, input_shape=(3, 224, 224), nb_classes=numberOfClasses)


In [8]:
# Step 4: Train the ART classifier

#classifier.fit(x_train, y_train, batch_size=64, nb_epochs=100)
#classifier.save("breakhis_squeezenet_64_100epochs", "C:\\Users\\Florian\\Documents\\PySyft-fork\\examples\\masterarbeit")

pathModel = "C:\\Users\\Florian\\Documents\\PySyft-fork\\examples\\masterarbeit\\breakhis_squeezenet_32_100epochs.model"
pathOptimizer = "C:\\Users\\Florian\\Documents\\PySyft-fork\\examples\\masterarbeit\\breakhis_squeezenet_32_100epochs.optimizer"

model.load_state_dict(torch.load(pathModel))
optimizer.load_state_dict(torch.load(pathOptimizer))

In [22]:
x_test[1:65].shape

(64, 3, 224, 224)

In [11]:
# Step 5: Evaluate the ART classifier on benign test examples

predictions = classifier.predict(x_test[1:65])
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)

print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 4.00 GiB total capacity; 2.50 GiB already allocated; 0 bytes free; 48.95 MiB cached)

In [ ]:
# generateEvasionLabels
#def generateEvasionLabels():
#    evasion_labels = np.zeros((len(labels.data.numpy()), len(breakhis.classes)))
#    
#    for i in range(len(evasion_labels)):
#        evasion_labels[i][1]=1
#    return evasion_labels

In [ ]:
# Step 6: Generate adversarial test examples
#attack = FastGradientMethod(classifier=classifier, eps=0.2)

#dictForAttacker ={
#    "targeted": True
#    #"theta": 0.3
#}
#, attacker_params=dictForAttacker

#delta = desired accuracy on perturbed samples
attack = UniversalPerturbation(classifier, attacker="deepfool", delta=0.25, max_iter=100, eps=45, norm=2)
#attack = UniversalPerturbation(classifier, attacker="newtonfool", delta=0.25, max_iter=100, eps=1000, norm=2)

x_test_adv = attack.generate(x=x_adv)

In [ ]:
# Step 7: Test adversarial samples
predictions = classifier.predict(x_test_adv)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_adv, axis=1)) / len(y_adv)
print('Accuracy on adversarial test examples: {}%'.format(accuracy * 100))

In [ ]:
# Step 7: Test benign samples
predictions = classifier.predict(x_benign)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_benign, axis=1)) / len(y_benign)
print('Accuracy on benign test examples: {}%'.format(accuracy * 100))

In [ ]:
plt.imshow(x_adv[1].transpose((1, 2, 0)))
plt.show()

In [ ]:
plt.imshow(x_test_adv[1].transpose((1, 2, 0)))
plt.show()

In [ ]:
pixelpattern = x_adv[2].transpose((1, 2, 0))-x_test_adv[2].transpose((1, 2, 0))
plt.axis('off')
plt.imshow(pixelpattern)
plt.savefig('test.png',bbox_inches='tight', pad_inches = 0)

In [ ]:
x_adv[1].transpose((1, 2, 0))-x_test_adv[1].transpose((1, 2, 0)) == x_adv[2].transpose((1, 2, 0))-x_test_adv[2].transpose((1, 2, 0))

In [ ]:
x_adv[2].transpose((1, 2, 0))-x_test_adv[2].transpose((1, 2, 0))

In [ ]:
#Rescale to 0-255 and convert to uint8
rescaled = (255.0 / pixelpattern.max() * (pixelpattern - pixelpattern.min())).astype(np.uint8)
im = Image.fromarray(rescaled)
im.save('test1.png')

3